# Entrenamiento y Evaluación de Modelos

## Trabajo Práctico Nro. 2 - Grupo 3

#### Integrantes:
* Ignacio Busso
* Lucas Copes
* Jesica Heit

#### Dataset: https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction
* Detalle: contiene datos de la satisfacción de los pasajeros de diferentes vuelos tomando en cuenta multiples aspectos (calidad del servicio, comodidad, limpieza, etc.)
* Target: columna 'satisfaction', para determinar la satisfacción de un pasajero respecto a un vuelo.
* Dimensiones: 25 columnas x 129.880 filas.

In [ ]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

pd.options.display.max_columns = 0

#Cambios en el estilo de los graficos
plt.style.use('fast')

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [ ]:
# Lectura y concatenación de los .csv
train = pd.read_csv('data/train.csv', index_col=[0])
test = pd.read_csv('data/test.csv', index_col=[0])
all_data = pd.concat([train, test], sort=False)

# Asignamos nuevos nombres a algunas de las columnas
new_column_names = {
    'Gender': 'gender',
    'Customer Type': 'customer_type',
    'Age': 'age',
    'Type of Travel': 'business_travel',
    'Class': 'ticket_class',
    'Flight Distance': 'flight_distance',
    'Inflight wifi service': 'wifi_service',
    'Departure/Arrival time convenient': 'departure_arrival_time_convenient',
    'Ease of Online booking': 'online_booking',
    'Gate location': 'gate_location',
    'Food and drink': 'food_and_drink',
    'Online boarding': 'online_boarding',
    'Seat comfort': 'seat_comfort',
    'Inflight entertainment': 'inflight_entertainment',
    'On-board service': 'onboard_service',
    'Leg room service': 'leg_room',
    'Baggage handling': 'baggage_handling',
    'Checkin service': 'checkin',
    'Inflight service': 'inflight_service',
    'Cleanliness': 'cleanliness',
    'Departure Delay in Minutes': 'departure_delay',
    'Arrival Delay in Minutes': 'arrival_delay',
}

all_data.rename(columns=new_column_names, inplace=True)
all_data.set_index('id', inplace=True)

## Métrica

F-Score es una medida de precisión de un modelo de datos. Se utiliza para evaluar los sistemas de clasificación binaria, estos se clasifican en positivos o negativos.
Combina precission (se refiere al número de verdaderos positivos dividido por el número total de predicciones positivas (el número de verdaderos positivos más el número de falsos positivos)) y recall (calcular los elementos relevantes (el número de verdaderos positivos dividido el número de elementos relevantes)
F-Score es la media armónica entre precission y recall.

Cuando F-Score tiende a estar más cercano al 1 quiere decir que precission y recall son muy buenos, en cambio, cuando tiende a 0 quiere decir que ambos métodos son muy malos

## Preprocesado
1. Conversión de variables cualitativas a booleanas cuantitativas.
2. 
    - Limpieza de registros con nulos implícitos en los servicios. Aplica solo a features con menos de 500 casos.
    - Limpieza de registros NaN en `arrival_delays`. (~400 registros)
3. Limpieza de valores extremos en `arrival_delays` y `departure_delays`. (~600 registros)
4. Limpieza de valores extremos en `flight_distance`. Aplica solo a vuelos de más de 4000 unidades de longitud (~75 registros)
5. Feature Engineering creando intervalos para cuatro variables numéricas. (`age`, `flight_distance`, `departure_delay` y `arrival_delay`)

In [ ]:
# Conversión a variables booleanas
all_data['gender'] = all_data['gender'].replace(['Male','Female'],[0,1])
all_data['customer_type'] = all_data['customer_type'].replace(['disloyal Customer','Loyal Customer'],[0,1])
all_data['business_travel'] = all_data['business_travel'].replace(['Personal Travel','Business travel'],[0,1])
all_data['satisfaction'] = all_data['satisfaction'].replace(['neutral or dissatisfied','satisfied'],[0,1])

# Limpieza de filas con pocas (< 500) features de servicios nulas (== 0) y arrivals NaNs
all_data = all_data[
    ~(all_data == 0).gate_location &
    ~(all_data == 0).food_and_drink &
    ~(all_data == 0).seat_comfort & 
    ~(all_data == 0).inflight_entertainment &
    ~(all_data == 0).onboard_service &
    ~(all_data == 0).checkin &
    ~(all_data == 0).inflight_service &
    ~(all_data == 0).cleanliness &
    ~all_data.arrival_delay.isnull()
    ]

# Limpieza de valores extremos en Arrivals y Departures
all_data = all_data.drop(all_data[all_data.arrival_delay > 240].index)
all_data = all_data.drop(all_data[all_data.departure_delay > 240].index)

# Limpieza de valores extremos en flight_distance (> 4000 - 75 registros)
all_data = all_data.drop(all_data[all_data.flight_distance > 4000].index)

# Creación de intervalos para features numéricas
    # Puede ser visto como Binning / Redondeo de números? Feature Engineering?
    # Ver de dejarlo para el final. Ejecutarlo por separado en otro df y comparar resultados.

# ¿Corresponde que los labels tengan como nombre el mayor numero del intervalo? ¿Debería usar un valor calculado en su lugar? (Media/Mediana/Moda)
all_data["age_interval"] = pd.cut(x=all_data['age'], bins=[0,12,18,30,60,80,100], labels=[12,18,30,60,80,100])
all_data["flight_distance_interval"] = pd.cut(x=all_data['flight_distance'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=[500,1000,1500,2000,2500,3000,3500,4000,4500,5000])
all_data["departure_delay_interval"] = pd.cut(x=all_data['departure_delay'], bins=[-1,0,15,30,45,60,120,180,240], labels=[0,15,30,45,60,120,180,240])
all_data["arrival_delay_interval"] = pd.cut(x=all_data['arrival_delay'], bins=[-1,0,15,30,45,60,120,180,240], labels=[0,15,30,45,60,120,180,240])

### Armado de datasets
- 60% Train         (77k)
- 20% Validation    (25k)
- 20% Test          (25k)

In [ ]:
from sklearn.model_selection import train_test_split

train, not_train = train_test_split(all_data, test_size=0.4, random_state=7)
validation, test = train_test_split(not_train, test_size=0.5, random_state=7)

train.shape, validation.shape, test.shape

### DataFrameMapper
- Escalar todas las variables numéricas no booleanas con MinMaxScaler (valores entre 0-1).
- Imputar las features con más de 600 nulos con IterativeImputer.

In [ ]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Imputar y escalar
mapper = DataFrameMapper([
    (['gender'], None),
    (['customer_type'], None),
    (['age'], [MinMaxScaler()]),
    (['business_travel'], None),
    (['ticket_class'], [OneHotEncoder()]),
    (['flight_distance'], [MinMaxScaler()]),
    (['wifi_service'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['departure_arrival_time_convenient'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['online_booking'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['gate_location'], [MinMaxScaler()]),
    (['food_and_drink'], [MinMaxScaler()]),
    (['online_boarding'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['seat_comfort'], [MinMaxScaler()]),
    (['inflight_entertainment'], [MinMaxScaler()]),
    (['onboard_service'], [MinMaxScaler()]),
    (['leg_room'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['baggage_handling'], [MinMaxScaler()]),
    (['checkin'], [MinMaxScaler()]),
    (['inflight_service'], [MinMaxScaler()]),
    (['cleanliness'], [MinMaxScaler()]),
    (['departure_delay'], [MinMaxScaler()]),
    (['arrival_delay'], [MinMaxScaler()]),
], df_out=True)
mapper.fit(train)

fe_mapper = DataFrameMapper([
    (['gender'], None),
    (['customer_type'], None),
    (['age_interval'], [MinMaxScaler()]),
    (['business_travel'], None),
    (['ticket_class'], [OneHotEncoder()]),
    (['flight_distance_interval'], [MinMaxScaler()]),
    (['wifi_service'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['departure_arrival_time_convenient'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['online_booking'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['gate_location'], [MinMaxScaler()]),
    (['food_and_drink'], [MinMaxScaler()]),
    (['online_boarding'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['seat_comfort'], [MinMaxScaler()]),
    (['inflight_entertainment'], [MinMaxScaler()]),
    (['onboard_service'], [MinMaxScaler()]),
    (['leg_room'], [IterativeImputer(missing_values=0), MinMaxScaler()]),
    (['baggage_handling'], [MinMaxScaler()]),
    (['checkin'], [MinMaxScaler()]),
    (['inflight_service'], [MinMaxScaler()]),
    (['cleanliness'], [MinMaxScaler()]),
    (['departure_delay_interval'], [MinMaxScaler()]),
    (['arrival_delay_interval'], [MinMaxScaler()]),
], df_out=True)
fe_mapper.fit(train)

#mfit = mapper.fit(all_data)
#all_data_transformed = mapper.transform(all_data)

In [ ]:
# Transformación de un sample:
sample = train.sample(7, random_state=7)
# Sample original:
sample

In [ ]:
# Sample transformado.
mapper.transform(sample)

## Modelos
Serán utilizados los siguientes algoritmos:
1. Regresión Logística
2. Arboles de Decisión
3. Gradient Boosting

In [ ]:
from collections import defaultdict
from sklearn import metrics

import seaborn as sns

# Funcion Fisa
def evaluate_model(model, set_names=('train', 'validation'), title='', show_cm=True):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
    
    if show_cm:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]  # <- hack feo...

        y = set_data.satisfaction
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
        if show_cm:
            ax = axis[i]
            sns.heatmap(metrics.confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['not happy', 'happy'])
            ax.yaxis.set_ticklabels(['not happy', 'happy'])
            ax.set_xlabel('Predicted class')
            ax.set_ylabel('True class')

    display(pd.DataFrame(final_metrics, index=set_names))
    if show_cm:
        plt.tight_layout()
        plt.show()

### Regresión Logística

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

#### Regresión Logística - Parámetros estándar

In [ ]:
regression_model = LogisticRegression(random_state=7)
lr_model = Pipeline([
    ('mapper', mapper),
    ('classifier', regression_model),
])

lr_model.fit(train, train.satisfaction)
evaluate_model(lr_model, title='Regresión Lineal')

#### Regresión Logística - Iteraciones máximas

Entrenamiento limitando la cantidad de iteraciones máximas que el algoritmo puede hacer antes de converger.

In [ ]:
regression_model = LogisticRegression(random_state=7, max_iter=2)
lr_model = Pipeline([
    ('mapper', mapper),
    ('classifier', regression_model),
])

lr_model.fit(train, train.satisfaction)
evaluate_model(lr_model, title='Regresión Lineal - max_iter = 2')

La métrica Accuracy cae ~12 puntos porcentuales en este caso, ya que al limitar la cantidad de iteraciones forzamos al algoritmo a converger antes de tiempo, cuando todavía hay lugar para mejores resultados.

### Árboles de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Función para graficar los arboles:

In [ ]:
import graphviz  # pip install graphviz
from sklearn.tree import export_graphviz

def graph_tree(tree, col_names):
    graph_data = export_graphviz(
        tree, 
        out_file=None, 
        feature_names=col_names,  
        class_names=['Neutral/Not Satisfied', 'Satisfied'],  
        filled=True, 
        rounded=True,  
        special_characters=True,
    )
    graph = graphviz.Source(graph_data)  
    return graph

#### Árboles de Decisión - Parámetros estándar

In [ ]:
tree_model = DecisionTreeClassifier(random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Decision Tree')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

El resultado del entrenamiento sin definir parámetros es un overfitting bastante claro. Esto se debe principalmente al hecho de que no se le ha definido una profundidad máxima, permitiendo así que el arbol siga memorizando los ejemplos y creando una rama para cada uno de ellos.

#### Árboles de Decisión - Profundidad máxima
Entrenamiento definiendo un `max_depth` de 4 (Default = None). De esta manera buscamos eliminar el overfitting visto en el caso anterior.

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=4,random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Decision Tree - max_depth = 4')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

Ya con un límite en la cantidad máxima de niveles permitida, se alcanzan valores más realistas.

Entrenamiento definiendo un `max_depth` de 1 (valor mínimo).

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=1,random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Decision Tree - max_depth = 1')

print('Cantidad de nodos:', tree_model.tree_.node_count)
print('Profundidad máxima:', tree_model.tree_.max_depth)

Utilizar el valor mínimo de profundidad acarrea a que el Accuracy caiga 8 puntos porcentuales. Naturalmente, esto ocurre ya que acotamos demasiado la cantidad de cortes que puede hacer el árbol de decisión.
En el siguiente gráfico podemos ver que la clasificación se hace solamente en base al valor de la feature `online_boarding`:

In [ ]:
graph_tree(tree_model, mapper.transformed_names_)

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

#### Gradient Boosting - Entrenamiento con parámetros estándar

In [ ]:
gb_model = GradientBoostingClassifier(random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gb_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Gradient Boosting')

#### Gradient Boosting - Learning rate
Entrenamiento con un learning rate mucho mayor (20, en lugar del 0.1 por default).

In [ ]:
gb_model = GradientBoostingClassifier(learning_rate=20, random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gb_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Gradient Boosting - learning_rate = 20')

Esto resulta en una métrica Accuracy mucho más baja que la anterior, ya que el modelo falla y clasifica erroneamente a muchos de los casos. Por otro lado, el tiempo de ejecución se ve afectado solo mínimamente.

#### Gradient Boosting - Cantidad de árboles
Entrenamiento con un `n_estimators` mucho menor (2, en lugar de los 100 por default). Este parámetro indica la cantidad de arboles que Gradient Boosting va a modelar.

In [ ]:
gb_model = GradientBoostingClassifier(n_estimators=2, random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gb_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Gradient Boosting - n_estimators = 2')

Esto resulta en una disminución de aproximadamente 12 puntos porcentuales en la métrica Accuracy al momento de predecir la variable target (comparado con la ejecución por default). Al limitar la cantidad de arboles de Gradient Boosting estamos afectando una de sus principales fortalezas: la de aprender de los errores del modelo anterior. 

Por eso mismo este ejemplo falla más al momento de predecir, y también tiene un Recall considerablemente más bajo al original, lo que muestra que con solo dos árboles existe todavía una cantidad considerable de falsos negativos.

In [ ]:
gb_model = GradientBoostingClassifier(n_estimators=3, random_state=7)
dt_model = Pipeline([
    ('mapper', mapper),
    ('classifier', gb_model),
])

dt_model.fit(train, train.satisfaction)
evaluate_model(dt_model, title='Gradient Boosting - n_estimators = 3')

Con respecto al modelo anterior, al añadir un tercer árbol ya el Recall sube 20 puntos porcentuales (eliminando parte considerable de los falsos negativos), mientras que el Accuracy sube 6.